In [ ]:
import timeit

start = timeit.default_timer()

In [ ]:
import pandas as pd
import os
import numpy as np
import geopandas as gpd
import fiona
import wget
import rasterio
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import glob
import zipfile
import time
#from datetime import datetime, timedelta
import datetime

import rioxarray
from rasterio import plot
from rasterio import mask
from rasterio.mask import mask
import xarray 
from pathlib import Path
import matplotlib.pyplot as plt

import re
import requests
#pd.set_option('display.max_columns', None)
#pd.set_option('max_colwidth', None)

from osgeo import gdal, osr
from rasterio.warp import reproject, Resampling


from shapely.geometry import Polygon

# Creating and plotting the flowchart libs:
import pygraphviz as pgv
from PIL import Image

# Library used to get all folders in a given directory
# It is used in the removal process
import shutil

# stop warning:
import warnings
warnings.filterwarnings("ignore")

import math
# adding a background map
import contextily as ctx

from tqdm import tqdm

In [ ]:
# Importing the area of interest
aoi = gpd.read_file("./shapefiles/area_of_intrest.shp")

# Transform to WGS84
aoi_wgs = aoi.to_crs('EPSG:4326')

In [ ]:
# creating sentinel api
api = SentinelAPI('UserName','Password','https://apihub.copernicus.eu/apihub/')

In [ ]:
# create a footprint of the data
footprint = None
for i in aoi_s_wgs['geometry']:
    footprint = i


# entering the required start date
start_date = '2019-01-01'

# entering the end date
end_date = '2023-04-10'

# changing the dates format
api_start = start_date.translate( { ord("-"): None } )
api_end = end_date.translate( { ord("-"): None } )

# extrating the all available products:
satellite_data = api.query(
    footprint,
    date =(api_start, api_end),
    platformname = 'Sentinel-2',
    processinglevel = 'Level-2A',
    cloudcoverpercentage = (0,10))

In [ ]:
# convering the results to a geodataframe for further filtration 
products = api.to_geodataframe(satellite_data)

# 
products_aoi = products[products['filename'].str.contains("R120_T40QEM")]
products_aoi = products_aoi[products_aoi['platformserialidentifier'] == 'Sentinel-2A']
products_aoi['date'] = pd.to_datetime(products_aoi['beginposition']).dt.date

In [ ]:
# extrating the products dates to a list
dates_list = pd.to_datetime(products_aoi['beginposition']).dt.date.tolist()

# using the input dates, generate dates that are 3 months apart:
required_dates = pd.date_range(start_date,
                    end_date,
                    freq='10d').strftime('%Y-%m-%d').tolist()



In [ ]:
# format the dates 
formulated_dates =[]
for date in required_dates:
    try:
        read = datetime.datetime.strptime(date,'%Y-%m-%d').date()
        formulated_dates.append(read)
    except ValueError:
        print("ValueError in: ")
        print(date)

In [ ]:
# selecting the dates in the products df that are nearest to the dates of the entered dates.
selected_dates = []
for date in formulated_dates:
    selected_dates.append(min(dates_list , key=lambda sub: abs(sub - date)))
selected_dates = list(set(selected_dates))
selected_dates.sort()
selected_dates

In [ ]:
# extracting the data based on the dates:
scenes = products_aoi[products_aoi['date'].isin(selected_dates)].to_dict('records')
scenes

In [ ]:
# create a downloads file in the path, if it does not exist
download_dir = os.path.join(os.getcwd() , 'downloads')
os.makedirs(download_dir, exist_ok=True)


In [ ]:
# les that are part of the current analysis:
files_to_delete = [scene['title'] + ext for scene in scenes for ext in ['.zip', '.SAFE']]


def delete_files_and_folders_not_in_list(file_list):    
    # Get a list of all files and folders in the directory
    all_items = os.listdir(download_dir)
    
    # Iterate over each item in the directory
    for item_name in all_items:
        # Create the item path
        item_path = os.path.join(download_dir, item_name)
        
        # Check if the item is not in the provided list
        if item_name not in file_list:
            # Check if the item path is a file
            if os.path.isfile(item_path):
                # Delete the file
                os.remove(item_path)
                print(f"Deleted file: {item_path}")
            elif os.path.isdir(item_path):
                # Delete the folder and its contents
                shutil.rmtree(item_path)
                print(f"Deleted folder: {item_path}")


# Call the function to delete files not in the list
delete_files_and_folders_not_in_list(files_to_delete)

In [ ]:
# Loop through each scene id and attempt to download, try twice before moving to the next one
downloaded_basenames = [os.path.basename(downloaded_file) for downloaded_file in glob.glob(os.path.join(download_dir, '**', '*.zip'), recursive=True)]
downloaded_scene_titles = {os.path.splitext(downloaded_basename)[0] for downloaded_basename in downloaded_basenames}

all_scene_titles = {scene['title'] for scene in scenes}

scene_ids = [scene['uuid'] for scene in scenes if scene['title'] in all_scene_titles.difference(downloaded_scene_titles)]
# print("[INFO] {} scenes remain to download".format(len(scene_ids)))


for idx, scene_id in enumerate(scene_ids):
    product_info = api.get_product_odata(scene_id)
    print("[INFO] Product {} ({}/{}); Online: {}".format(
        product_info['id'],
        idx + 1,
        len(scene_ids),
        product_info['Online']))
    attempts = 0
    
    while attempts < 1:
        try:
            api.download(product_info['id'], download_dir)
            break  # If successful, move past
        except Exception as e:
            print("[ERROR] Request failed.. (current time: {})".format(
                datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))
        attempts += 1




# update the list after the initial request to select any scenes that were not downloaded
downloaded_basenames = [os.path.basename(downloaded_file) for downloaded_file in glob.glob(os.path.join(download_dir, '**', '*.zip'), recursive=True)]
downloaded_scene_titles = {os.path.splitext(downloaded_basename)[0] for downloaded_basename in downloaded_basenames}
scene_ids = [scene['uuid'] for scene in scenes if scene['title'] in all_scene_titles.difference(downloaded_scene_titles)]

# this loop wait 30 minutes after the initial downloading so that scenes become online 
if len(scene_ids) == 0:
    print("[INFO] No more files left to download from CSV files")
else:
    print("[INFO] {} scenes remain to download".format(len(scene_ids)))
    print('[INFO] Sleeping for 31 minutes until the files are online to download (downloading will resume at: {})'.format(
                    (datetime.datetime.now() + datetime.timedelta(minutes=31)).strftime("%Y-%m-%d %H:%M") ))
    time.sleep(60 * 30)
    for idx, scene_id in enumerate(scene_ids):
        product_info = api.get_product_odata(scene_id)
        print("[INFO] Product {} ({}/{}); Online: {}".format(
            product_info['id'],
            idx + 1,
            len(scene_ids),
            product_info['Online']))
        attempts = 0
        
        while attempts < 10:
            try:
                api.download(product_info['id'], download_dir)
                break  # If successful, move past
            except Exception as e:
                wait_time = 2
                print("[ERROR] Request failed.. waiting {} seconds before retrying (current time: {})".format(
                    wait_time * 60,
                    datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))
                time.sleep(60 * wait_time)
            attempts += 1

In [ ]:
# extracting all the downloade files:
def un_zipFiles(download_dir):
    files=os.listdir(download_dir)
    for file in files:
        if file.endswith('.zip'):
            filePath=download_dir+'/'+file
            zip_file = zipfile.ZipFile(filePath)
            for names in zip_file.namelist():
                zip_file.extract(names,download_dir)
            zip_file.close() 
            
un_zipFiles(download_dir)

In [ ]:
aoi_polygons = list(aoi.geometry)
aoi_polygons

In [ ]:
# this function import the band, and filter it to remove any saturated pixels
def importing_jp2_before(item,list):
    image_name = item.name.split('.')[0]
    list.append(image_name)
    xds = rioxarray.open_rasterio(os.path.join(download_dir, item))
    xds = xds.where(xds > 1, other = 1)
    xds = xds.where(xds < 10000, other = 1)
    xds = xds.rio.clip(aoi_polygons, aoi.crs, drop=True).to_dataset(name ='band_data')
    globals()[image_name] = xds

In [ ]:
Band3_list = []   # Green
Band4_list = []   # Red
Band8_list = []   # NIR

# importing the required scenes and adding their names to the lists above
for item in Path(download_dir).glob('**/*10m.jp2'):
    image_date = item.name.split('_')[1].split('T')[0]
    image_date = datetime.datetime.strptime(image_date, "%Y%m%d").strftime("%Y-%m-%d")
    if item.name.endswith('B03_10m.jp2'):
        importing_jp2_before(item,Band3_list)
    if item.name.endswith('B04_10m.jp2'):
        importing_jp2_before(item,Band4_list)
    if item.name.endswith('B08_10m.jp2'):
        importing_jp2_before(item,Band8_list)

In [ ]:
# si_9 = (nir*red)/green

# generating the soil salinity index 9
si_9_result = []
for index, band in enumerate(selected_dates):
    date_name = format(band.strftime('%Y%m%d'))
    file_name = "si_9_{}".format(date_name)
    band_4 = locals()[[i for i in vars() if date_name in i and 'B04' in i][0]]
    band_8 = locals()[[i for i in vars() if date_name in i and 'B08' in i][0]] 
    band_3 = locals()[[i for i in vars() if date_name in i and 'B03' in i][0]]
    if selected_dates[index] < datetime.datetime.strptime('2022-01-25', '%Y-%m-%d').date():
        results = ((band_4) * (band_8)) / (band_3)
    elif selected_dates[index] >= datetime.datetime.strptime('2022-01-25', '%Y-%m-%d').date():
        band_4_e = band_4 - 1000
        band_8_e = band_8 - 1000
        band_3_e = band_3 - 1000
        band_4_e = band_4_e.where(band_4_e['band_data'] > 1, other = 1)
        band_8_e = band_8_e.where(band_8_e['band_data'] > 1, other = 1)
        band_3_e = band_3_e.where(band_3_e['band_data'] > 1, other = 1)
        results= ((band_4_e * band_8_e) / band_3_e)
    results = results.where(results > 1, other = 1)
    results = results.where(results < 500, other = 1)
    results = results.rio.clip(aoi_polygons, aoi.crs, drop=True)
    print(results.band_data.mean().item())
    results["time"] = np.datetime64(band)
    results = results.rename_vars({'band_data': 'index_9'})
    globals()[file_name] = results
    si_9_result.append(vars()[f"{file_name}"])
    results.to_netcdf('C:/Users/G/Desktop/Diss/soil_salinity/full_resolution/' + file_name + '.nc4', engine="netcdf4", encoding = {"index_9": {"dtype": "float64"}})


si_9_results = xarray.concat(si_9_result, dim="time")
si_9_results.to_netcdf('C:/Users/G/Desktop/Diss/soil_salinity/ss_9.nc4', engine="netcdf4")
    

In [ ]:
# generating the soil salinity index 9

NDVI_result = []
for band in selected_dates:
    date_name = format(band.strftime('%Y%m%d'))
    file_name = "ndvi_{}".format(date_name)
    band_4 = locals()[[i for i in vars() if date_name in i and 'B04' in i][0]]
    band_8 = locals()[[i for i in vars() if date_name in i and 'B08' in i][0]] 
    if selected_dates[index] < datetime.datetime.strptime('2022-01-25', '%Y-%m-%d').date():
        results = ((band_8) - (band_4)) / ((band_8) + (band_4))
    elif selected_dates[index] >= datetime.datetime.strptime('2022-01-25', '%Y-%m-%d').date():
        band_4_mean = band_4.band_data.mean().item()
        band_8_mean = band_8.band_data.mean().item()
        band_4_e = band_4 - 1000
        band_8_e = band_8 - 1000
        band_4_e = band_4_e.where(band_4_e['band_data'] > 1, other = 1)
        band_8_e = band_8_e.where(band_8_e['band_data'] > 1, other = 1)
        results = (band_8 - band_4 ) / (band_8  + band_4)
    results = results.where(results >= -1, other = 0)
    results = results.where(results <= 1, other = 0)
    results = results.rio.clip(aoi_polygons, aoi.crs, drop=True)
    print(results.band_data.mean().item())
    results = results.rename_vars({'band_data': 'ndvi'})
    results["time"] = np.datetime64(band)
    globals()[file_name] = results
    results.to_netcdf('C:/Users/G/Desktop/Diss/NDVI/full_resolution/' + file_name + '.nc4', engine="netcdf4")
    NDVI_result.append(vars()[f"{file_name}"])



NDVI_results = xarray.concat(NDVI_result, dim="time")
NDVI_results.to_netcdf('C:/Users/G/Desktop/Diss/NDVI/NDVI.nc4', engine="netcdf4")

In [ ]:
stop = timeit.default_timer()

print('Time: ', stop - start) 

